In [1]:
import pandas as pd
import numpy as np

In [2]:
import tqdm

In [3]:
baseData = pd.read_csv("../data/train_ver2.csv")

C:\Users\caose\AppData\Local\Temp\ipykernel_70500\3097669333.py:1: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  baseData = pd.read_csv("../data/train_ver2.csv")


In [4]:
### 3개월 데이터로 줄인다. 값이 너무 많아서 최근의 추세로 판단하겠다

In [5]:
base1 = baseData.loc[baseData.fecha_dato == "2016-04-28"]

In [6]:
base2 = baseData.loc[baseData.fecha_dato == "2016-05-28"]

In [7]:
data1 = base1.append(base2, ignore_index=True)

C:\Users\caose\AppData\Local\Temp\ipykernel_70500\4183799191.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data1 = base1.append(base2, ignore_index=True)


In [8]:
data1 = data1.reset_index(drop=True)

In [9]:
# 기존
data1 = data1.drop(['fecha_dato', 'ncodpers','ind_empleado','pais_residencia','fecha_alta','ind_nuevo','indrel','ult_fec_cli_1t',\
                    'conyuemp','canal_entrada','indfall','tipodom','cod_prov','nomprov','ind_actividad_cliente',\
                    'ind_aval_fin_ult1','ind_ctma_fin_ult1','ind_ctop_fin_ult1',\
                    'ind_ctpp_fin_ult1','ind_ecue_fin_ult1','ind_hip_fin_ult1','indresi',\
                    'ind_reca_fin_ult1','ind_viv_fin_ult1','ind_nomina_ult1','ind_nom_pens_ult1','ind_recibo_ult1',\
                    'ind_cco_fin_ult1','ind_cder_fin_ult1',"antiguedad",'indrel_1mes','tiprel_1mes',\
                    'ind_deco_fin_ult1','ind_deme_fin_ult1','ind_ahor_fin_ult1','ind_plan_fin_ult1','ind_pres_fin_ult1'], axis=1)

In [10]:
oneStepDf = data1.copy()

# 칼럼 처리

### 나이

In [11]:
oneStepDf["age"] = oneStepDf["age"].astype(int)

In [12]:
oneStepDf["age"] = np.where(oneStepDf.age <= 9,0,
                   np.where(oneStepDf.age <= 19,1,
                   np.where(oneStepDf.age <= 29,2,
                   np.where(oneStepDf.age <= 39,3,
                   np.where(oneStepDf.age <= 49,4,
                   np.where(oneStepDf.age <= 59,5,
                   np.where(oneStepDf.age <= 69,6,
                   np.where(oneStepDf.age <= 79,7,
                   np.where(oneStepDf.age <= 89,8,
                   np.where(oneStepDf.age <= 99,9,10))))))))))

### 총소득

In [13]:
oneStepDf["renta"].fillna(oneStepDf.groupby('age')["renta"].transform('median'), inplace=True)

In [14]:
oneStepDf["renta"] = oneStepDf["renta"].astype(int)

In [15]:
oneStepDf["renta"].describe().astype(int)

count     1859727
mean       125652
std        202483
min          1202
25%         78250
50%         96544
75%        133053
max      28894395
Name: renta, dtype: int32

In [ ]:
oneStepDf["renta"] = np.where(oneStepDf.renta <= oneStepDf['renta'].quantile(0.2), 1,
                     np.where(oneStepDf.renta <= oneStepDf['renta'].quantile(0.4), 2,
                     np.where(oneStepDf.renta <= oneStepDf['renta'].quantile(0.6), 3,
                     np.where(oneStepDf.renta <= oneStepDf['renta'].quantile(0.8), 4, 5))))

In [ ]:
oneStepDf["renta"].drop_duplicates()

### 성별

In [ ]:
oneStepDf.sexo.drop_duplicates()

In [ ]:
oneStepDf.sexo = oneStepDf.sexo.fillna("H")

In [ ]:
oneStepDf.sexo.drop_duplicates()

In [ ]:
dict = {"V":0, "H":1}

oneStepDf.sexo = oneStepDf.sexo.str.upper()
oneStepDf.sexo = oneStepDf.sexo.map(dict)

In [ ]:
oneStepDf.sexo = oneStepDf.sexo.astype(int)

In [ ]:
oneStepDf.sexo.dtype

### 외국인 지수

In [ ]:
dict = {"N":0, "S":1}

oneStepDf.indext = oneStepDf.indext.str.upper()
oneStepDf.indext = oneStepDf.indext.map(dict)

In [ ]:
oneStepDf.indext.unique()

### 고객 분류

In [ ]:
oneStepDf.segmento.drop_duplicates()

In [ ]:
oneStepDf.segmento = np.where(oneStepDf.segmento == '03 - UNIVERSITARIO', 0, 
                     np.where(oneStepDf.segmento == '01 - TOP', 1,
                     np.where(oneStepDf.segmento == '02 - PARTICULARES', 1, oneStepDf.segmento)))

In [ ]:
oneStepDf.loc[oneStepDf["age"] < 2, 'segmento'] = oneStepDf.segmento.fillna(0)

In [ ]:
oneStepDf.loc[oneStepDf["age"] >= 2, 'segmento'] = oneStepDf.segmento.fillna(1)

In [ ]:
oneStepDf.segmento = oneStepDf.segmento.astype(int)

In [ ]:
oneStepDf.segmento.dtype

In [ ]:
oneStepDf.dtypes

### 나이

In [ ]:
# 0 ~ 19 미성년
oneStepDf.loc[oneStepDf["age"] == 0 , "age"] = 1
oneStepDf.loc[oneStepDf["age"] == 1 , "age"] = 1

# 20 ~ 49 청년
oneStepDf.loc[oneStepDf["age"] == 2 , "age"] = 2
oneStepDf.loc[oneStepDf["age"] == 3 , "age"] = 2
oneStepDf.loc[oneStepDf["age"] == 4 , "age"] = 2

# 50 ~ 69 중년
oneStepDf.loc[oneStepDf["age"] == 5 , "age"] = 3
oneStepDf.loc[oneStepDf["age"] == 6 , "age"] = 3

# 70이후 노년
oneStepDf.loc[oneStepDf["age"] == 7 , "age"] = 4
oneStepDf.loc[oneStepDf["age"] == 8 , "age"] = 4
oneStepDf.loc[oneStepDf["age"] == 9 , "age"] = 4
oneStepDf.loc[oneStepDf["age"] == 10 , "age"] = 4

In [ ]:
oneStepDf = oneStepDf.reset_index(drop=True)

In [ ]:
# label이 될 컬럼을 상품명으로 변경

oneStepDf.loc[oneStepDf['ind_dela_fin_ult1'] == 1, 'ind_dela_fin_ult1'] = '예금'
oneStepDf.loc[oneStepDf['ind_cno_fin_ult1'] == 1, 'ind_cno_fin_ult1'] = '급여계좌'
oneStepDf.loc[oneStepDf['ind_ctju_fin_ult1'] == 1, 'ind_ctju_fin_ult1'] = '주니어계좌'
oneStepDf.loc[oneStepDf['ind_fond_fin_ult1'] == 1, 'ind_fond_fin_ult1'] = '펀드계좌'
oneStepDf.loc[oneStepDf['ind_tjcr_fin_ult1'] == 1, 'ind_tjcr_fin_ult1'] = '신용카드'
oneStepDf.loc[oneStepDf['ind_valo_fin_ult1'] == 1, 'ind_valo_fin_ult1'] = '증권'
# oneStepDf.loc[oneStepDf['ind_deco_fin_ult1'] == 1, 'ind_deco_fin_ult1'] = '단기예금'
# oneStepDf.loc[oneStepDf['ind_deme_fin_ult1'] == 1, 'ind_deme_fin_ult1'] = '중기예금'
# oneStepDf.loc[data1['ind_ahor_fin_ult1'] == 1, 'ind_ahor_fin_ult1'] = '저축계좌'
# oneStepDf.loc[oneStepDf['ind_plan_fin_ult1'] == 1, 'ind_plan_fin_ult1'] = '연금'
# oneStepDf.loc[oneStepDf['ind_pres_fin_ult1'] == 1, 'ind_pres_fin_ult1'] = '대출'

In [ ]:
oneStepDf

In [ ]:
oneStepDf.columns

In [ ]:
# 라벨부분 컬럼
expandColumn = ['ind_cno_fin_ult1', 'ind_ctju_fin_ult1', 'ind_dela_fin_ult1',\
                'ind_fond_fin_ult1', 'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1']

In [ ]:
# 컬럼
dataCol = ['sexo', 'age', 'indext', 'renta', 'segmento']

In [ ]:
oneStepDf

### 11개 상품 1개의 컬럼으로 수정

In [ ]:
wholeList = []
for i in tqdm.tqdm(range(len(oneStepDf))):
    for j in expandColumn:
        rowList = []
        rowList.append(oneStepDf[j][i])
        for k in dataCol:
            rowList.append(oneStepDf[k][i])
        wholeList.append(rowList)
secondStepDf = pd.DataFrame(wholeList)

In [ ]:
secondStepDf.rename(columns={0:"FINANCIALPRODUCT"}, inplace=True)
secondStepDf.rename(columns={1:"GENDER"}, inplace=True)
secondStepDf.rename(columns={2:"AGE"}, inplace=True)
secondStepDf.rename(columns={3:"CUST_TYPE"}, inplace=True)
secondStepDf.rename(columns={4:"INCOME"}, inplace=True)
secondStepDf.rename(columns={5:"FOREINERINDEX"}, inplace=True)

secondStepDf

In [ ]:
# 가입 안 된 데이터 삭제
secondStepDf = secondStepDf.loc[secondStepDf.FINANCIALPRODUCT != 0]
secondStepDf = secondStepDf.reset_index(drop=True)
secondStepDf

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
labelEn = LabelEncoder()

In [ ]:
secondStepDf.FINANCIALPRODUCT = labelEn.fit_transform(secondStepDf.FINANCIALPRODUCT)

In [ ]:
secondStepDf.corr()

In [ ]:
secondStepDf.FINANCIALPRODUCT = labelEn.inverse_transform(secondStepDf.FINANCIALPRODUCT)

In [ ]:
secondStepDf

In [ ]:
----------------------------------------------------------------------------------------------------------------

In [ ]:
### csv저장

In [ ]:
secondStepDf.to_csv("../test/Financial_Product_Recommendation.csv", index=False, mode ="w")